In [2]:
import pandas as pd
import requests
import string
import urllib
import datadotworld as dw
import os
from scrapy import Selector

In [107]:
# Data file URL
url = 'https://www.bls.gov/cew/downloadable-data-files.htm'

#Pull in page content
r = requests.get(url)

#Make Selector item to scrape
sel = Selector(text = r.text)

# Select annual average file list
sing_files = sel.xpath('//*[@id="cew_naics_data"]/tbody//td[7]//@href').extract()

,area_fips,area_title
4,1000,Alabama
76,2000,Alaska
127,4000,Arizona
147,5000,Arkansas
227,6000,California
290,8000,Colorado
359,9000,Connecticut
372,10000,Delaware
386,12000,Florida
459,13000,Georgia


In [ ]:
# Define list of state codes from BLS link and filter to code and name
# This will be a reference to filter the single files
state_list = pd.read_csv('https://data.bls.gov/cew/doc/titles/area/area_titles.csv')
state_list = state_list[state_list['area_title'].str.contains('-- Statewide',regex=False)]
state_list['area_title'] = state_list['area_title'].str.split(' --',n=1,expand=True)
state_list['area_fips'] = state_list['area_fips'].astype('int64')
state_list

In [146]:
# Define list of industries from BLS
industry_list = pd.read_csv('https://data.bls.gov/cew/doc/titles/industry/industry_titles.csv')

# Winnow down to just six-digit values
six_digit = industry_list[industry_list['industry_code'].str.len()==6]

# Identify only the newspaper-related code: 511110
news = (six_digit[six_digit['industry_title']
                  .str.contains('Newspaper publishers',regex=False)]
       )

all = (industry_list[industry_list['industry_title']
                     .str.contains('Total, all industries',regex=False)]
      )

news_all = pd.concat([news,all])

,industry_code,industry_title
1648,511110,NAICS 511110 Newspaper publishers
0,10,"10 Total, all industries"


In [148]:
id=0
dfs=[]
for x in range(0,len(sing_files)-1):
    
    qcew = (pd.read_csv(sing_files[id])
            .merge(state_list, how='inner', on='area_fips')
            .merge(news_all,how='inner',on='industry_code')
           )
    dfs.append(qcew)
    id+=1
    
df = pd.concat(dfs,sort=False,ignore_index=True).drop_duplicates().reset_index(drop = True)

In [149]:
df.to_csv('newspaper_jobs_and_totals_by_state.csv')